In [2]:
import numpy as np
import pandas as pd

from glob import glob
from os import path

In [5]:
ABCD_SSD_dists = pd.read_csv('abcd_data/SSD_dist_by_subj.csv')
ABCD_SSD_dists

,NARGUID,SSDDur,proportion
0,003RTV85,300.0,0.250000
1,003RTV85,250.0,0.183333
2,003RTV85,400.0,0.183333
3,003RTV85,350.0,0.166667
4,003RTV85,450.0,0.100000
...,...,...,...
62441,xxmy9wd8,100.0,0.183333
62442,xxmy9wd8,150.0,0.133333
62443,xxmy9wd8,200.0,0.116667
62444,xxmy9wd8,250.0,0.100000


In [77]:
indiv_SSRTs = pd.DataFrame()
for file in [i for i in glob('ssrt_metrics/indiv_*.csv') if 'linear' not in i and 'guess' not in i]:
    data =  pd.read_csv(file, index_col=0)
    gen_str = file.split('indiv_')[-1].replace('.csv', '')
    indiv_ssrt_df = pd.DataFrame(np.nan, index=ABCD_SSD_dists.NARGUID.unique(), columns=['vanilla', 'w_guesses', 'w_graded'])
    for sub in ABCD_SSD_dists.NARGUID.unique():
        curr_sub = ABCD_SSD_dists.query("NARGUID=='%s'" % sub)
        if curr_sub.SSDDur.max() <= 550: #KLUDGE WHILE ALL SSDS ARE BEING SIMULATED
            indiv_ssrt_df.loc[sub, :] = [0, 0, 0]
            for SSD in curr_sub.SSDDur:
                ssd_SSRTs = data.loc[data.SSD==SSD, ['SSRT', 'SSRT_w_guesses', 'SSRT_w_graded']].values[0]
                weight = curr_sub.loc[curr_sub.SSDDur==SSD, 'proportion'].values
                indiv_ssrt_df.loc[sub, :] +=  ssd_SSRTs * weight
    indiv_SSRTs = pd.concat([indiv_SSRTs, indiv_ssrt_df.rank().add_prefix('gen-%s_SSRT-' % gen_str)], 1)


In [79]:
indiv_SSRTs.corr()

,gen-vanilla_SSRT-vanilla,gen-vanilla_SSRT-w_guesses,gen-vanilla_SSRT-w_graded,gen-graded_mu_go_log_SSRT-vanilla,gen-graded_mu_go_log_SSRT-w_guesses,gen-graded_mu_go_log_SSRT-w_graded
gen-vanilla_SSRT-vanilla,1.000000,-0.995841,0.980978,-0.094358,-0.995128,0.984557
gen-vanilla_SSRT-w_guesses,-0.995841,1.000000,-0.960094,0.062913,0.993231,-0.955112
gen-vanilla_SSRT-w_graded,0.980978,-0.960094,1.000000,-0.827257,-0.973769,0.986294
gen-graded_mu_go_log_SSRT-vanilla,-0.094358,0.062913,-0.827257,1.000000,0.129605,-0.855596
gen-graded_mu_go_log_SSRT-w_guesses,-0.995128,0.993231,-0.973769,0.129605,1.000000,-0.984177
gen-graded_mu_go_log_SSRT-w_graded,0.984557,-0.955112,0.986294,-0.855596,-0.984177,1.000000


In [72]:
data.loc[data.SSD==SSD, ['SSRT', 'SSRT_w_guesses', 'SSRT_w_graded']].values

array([], shape=(0, 3), dtype=float64)

In [73]:
SSD

0.0

In [74]:
data

,SSRT,mean_SSD,p_respond,max_RT,mean_go_RT,mean_stopfail_RT,omission_count,omission_rate,go_acc,stopfail_acc,SSD,SSRT_w_guesses,SSRT_w_graded
0,439.0,50.0,0.377367,1351.0,539.522453,271.839523,0,0.0,NaN,NaN,50.0,292.268335,868.0
1,393.0,100.0,0.390753,1351.0,539.522453,303.589708,0,0.0,NaN,NaN,100.0,296.994804,679.0
2,355.0,150.0,0.433880,1351.0,539.522453,342.891014,0,0.0,NaN,NaN,150.0,296.284940,546.0
3,321.0,200.0,0.489767,1351.0,539.522453,389.128321,0,0.0,NaN,NaN,200.0,291.000000,472.0
4,304.0,250.0,0.595880,1351.0,539.522453,426.326376,0,0.0,NaN,NaN,250.0,287.000000,423.0
5,292.0,300.0,0.704367,1351.0,539.522453,455.995562,0,0.0,NaN,NaN,300.0,283.000000,398.0
6,285.0,350.0,0.800000,1351.0,539.522453,482.024435,0,0.0,NaN,NaN,350.0,281.000000,351.0
7,279.0,400.0,0.872107,1351.0,539.522453,499.253408,0,0.0,NaN,NaN,400.0,277.527413,343.0
8,278.0,450.0,0.924560,1351.0,539.522453,514.026455,0,0.0,NaN,NaN,450.0,278.000000,325.0
9,278.0,500.0,0.957973,1351.0,539.522453,521.818915,0,0.0,NaN,NaN,500.0,279.000000,302.0


In [75]:
file

'ssrt_metrics/indiv_guesses.csv'

In [76]:
indiv_SSRTs

,gen-vanilla_SSRT-vanilla,gen-vanilla_SSRT-w_guesses,gen-vanilla_SSRT-w_graded
003RTV85,523.5,5242.0,560.0
00CY2MDM,5348.5,350.5,NaN
00HEV6HB,4435.0,1963.0,NaN
00LJVZK2,NaN,NaN,NaN
00NPMHND,NaN,NaN,NaN
...,...,...,...
hkfm1ruj,NaN,NaN,NaN
hx1ru4hv,4484.0,1402.0,NaN
jf8w3pw6,4083.0,1517.0,NaN
x8k59,3492.0,2197.0,NaN
